In [1]:
from __future__ import print_function
import os
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from models import G, D, weights_init
from data import get_training_set, get_test_set
import torch.backends.cudnn as cudnn
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
#torch.backends.cudnn.enabled = False


In [2]:
import numpy as np

In [3]:

batchSize=10
testBatchSize=10
nEpochs=500000
threads=2
lamb=100
seed=187
cuda=True
beta1=0.5
lr=0.0002
ndf=16
ngf=16
input_nc=3
output_nc=3

dataset='hdr'
dataroot='all_batched_dataset_256/mat_files/'
path=dataroot

import os
import scipy.io
ls=os.listdir(path)
#print ls
xs=[]
ys=[]
#for idx,i in enumerate(ls):
#    if(idx%100==0):
#        print ((idx,'ls'))
#    im1 = scipy.io.loadmat(path+i)
#    im1=im1['im']
#    xs.append(im1.shape[0])
#    ys.append(im1.shape[1])
s1=(256)
s2=(256)
print (s1)
print (s2)
imageSize=(s1,s2,3)

##################
ls=ls[0:10]

#####################
lls=len(ls)

import numpy as np
print (lls*3*s1*s2)
#X=np.zeros((lls,3,s1,s2))
#Y=np.zeros((lls,3,s1,s2))
X=np.random.rand(lls,3,s1,s2)
Y=np.random.rand(lls,3,s1,s2)

print ('phew')
count=0
import Image
for idx,i in enumerate(ls):
    li=len(i)
    i=i[0:li-4]
    im1 = scipy.io.loadmat(path+i+'.mat')
    im2=np.asarray(Image.open('all_batched_dataset_256/image_files/'+i+'.png'))    
    im2=im2/255.0;
    im1=im1['im']

    X[count,0,:,:]=im1[:    , : ,0]
    X[count,1,:,:]=im1[ :    , : ,1]
    X[count,2,:,:]=im1[:    , : ,2]

    Y[count,0, :    , : ]=im2[ :    , : ,0]
    Y[count,1, :    , : ]=im2[ :    , : ,1]
    Y[count,2, :    , : ]=im2[ :    , : ,2]
    count=count+1
    if(idx%1000==0):
        print (idx)
print(X.shape)


if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

cudnn.benchmark = True

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)




256
256
1966080
phew
0
(10, 3, 256, 256)


In [7]:
X[0,1,255,24]


0.02459319494664669

In [4]:
import csv
with open( 'X.csv', "wb") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):    
                writer.writerows(X[i,j,:,:])
                


In [5]:
import csv
with open( 'Y.csv', "wb") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):    
                writer.writerows(Y[i,j,:,:])
                


In [4]:

def get_batch_train():
    import numpy.random
    pos=numpy.random.randint(0,X.shape[0],(batchSize))
    x1=X[pos]
    y1=Y[pos]
    x1=torch.FloatTensor(x1)
    y1=torch.FloatTensor(y1)
    return (x1), (y1)


x1,y1=get_batch_train()
print('===> Building model')
netG = G(input_nc, output_nc, ngf)
netG.apply(weights_init)
netD = D(input_nc, output_nc, ndf)
netD.apply(weights_init)



===> Building model


D (
  (conv1): Conv2d(6, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (batch_norm4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (batch_norm8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (leaky_relu): LeakyReLU (0.2, inplace)
  (sigmoid): Sigmoid ()
)

In [5]:
criterion=nn.MSELoss()
criterion_l1 = nn.L1Loss()
criterion_mse = nn.MSELoss()

real_A = torch.FloatTensor(batchSize, input_nc, s1, s2)
real_B = torch.FloatTensor(batchSize, output_nc, s1, s2)
label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

if cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    criterion = criterion.cuda()
    criterion_l1 = criterion_l1.cuda()
    criterion_mse = criterion_mse.cuda()
    real_A = real_A.cuda()
    real_B = real_B.cuda()
    label = label.cuda()


real_A = Variable(real_A)
real_B = Variable(real_B)
label = Variable(label)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))



In [6]:
outf='output_dilated_true/'
def train(epoch):
    for iteration in range(10):

        data, target = get_batch_train()
        #print (epoch,1)

        #data, target= torch.from_numpy(data), torch.from_numpy(target)
        data, target = data.float().cuda(), target.float().cuda()
        data, target = Variable(data,requires_grad=True), Variable(target,requires_grad=True)

        optimizer.zero_grad()
        
        


        # train with fake
        fake_b = netG(data)
        loss = torch.mean(torch.pow(fake_b - target ,2  ))
        loss.backward( )
        optimizer.step()

        if(iteration==9 and epoch % 5==0):
            import numpy as np
            ss=Y.shape[0]
            xyvap=np.random.randint(ss)
            #vutils.save_image(Y[xyvap],
            #        '%s/real_samples.png' % outf)

            ddv=X[xyvap:xyvap+1]
            #print (ddv.shape)
            ddv2=Variable(torch.from_numpy(ddv).float().cuda())

            ddv3=Y[xyvap:xyvap+1]
            #print (ddv.shape)
            ddv4=Variable(torch.from_numpy(ddv3).float().cuda())
            #print (ddv2)
            fake = netG(ddv2)
            #print (fake.data)
            #print (ddv4)

            vutils.save_image(fake.data,
                    '%s/samples_epoch_%03d.png' % (outf, epoch) , normalize=False)
            vutils.save_image(ddv4.data,
                    '%s/samples_epoch_%03d.png' % (outf, epoch+1), normalize=False)

    print(epoch,loss.data[0])
    if(epoch%10==0): 
        torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))



In [7]:


def test():
    avg_psnr = 0
    for batch in testing_data_loader:
        input, target = Variable(batch[0]), Variable(batch[1])
        if cuda:
            input = input.cuda()
            target = target.cuda()

        prediction = netG(input)
        mse = criterion_mse(prediction, target)
        psnr = 10 * log10(1 / mse.data[0])
        avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))


def checkpoint(epoch):
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")
    if not os.path.exists(os.path.join("checkpoint", dataset)):
        os.mkdir(os.path.join("checkpoint", dataset))
    net_g_model_out_path = "checkpoint/{}/netG_model_epoch_{}.pth".format(dataset, epoch)
    net_d_model_out_path = "checkpoint/{}/netD_model_epoch_{}.pth".format(dataset, epoch)
    torch.save(netG.state_dict(), net_g_model_out_path)
    torch.save(netD.state_dict(), net_d_model_out_path)
    print("Checkpoint saved to {}".format("checkpoint" + dataset))

for epoch in range(0, nEpochs + 1):
    print (epoch)
    train(epoch)
    #test()
    if epoch % 50 == 0:
        checkpoint(epoch)


0
0 0.363223046064
Checkpoint saved to checkpointhdr
1
1 0.476005285978
2
2 0.325557976961
3
3 0.395489066839
4
4 0.320150256157
5
5 0.444800972939
6
6 0.267515182495
7
7 0.434292316437
8
8 0.329205662012
9
9 0.285726964474
10
10 0.347567647696
11
11 0.276838988066
12
12 0.360912978649
13
13 0.357390105724
14
14 0.341111749411
15
15 0.290149688721
16
16 0.36040699482
17
17 0.37228217721
18
18 0.321394056082
19
19 0.317758202553
20
20 0.464227646589
21
21 0.314256668091
22
22 0.277570903301
23
23 0.253123641014
24
24 0.403587907553
25
25 0.419377744198
26
26 0.311236590147
27
27 0.420554786921
28
28 0.291298866272
29
29 0.392632126808
30
30 0.398420155048
31
31 0.341313481331
32
32 0.312137901783
33
33 0.42790555954
34
34 0.293485552073
35
35 0.239884346724
36
36 0.397655755281
37
37 0.377022385597
38
38 0.525163292885
39
39 0.411848962307
40
40 0.285943508148
41
41 0.336429655552
42
42 0.327953934669
43
43 0.36851555109
44
44 0.287648290396
45
45 0.354324907064
46
46 0.444353580475
47


KeyboardInterrupt: 